In [1]:
''' compare GFS against alongtrack SD wind - convert wind(SD 3.5m) to wind(SD 10m) '''
''' use coare3.6 to convert wind at 10m - need: uwnd, vwnd, air temp, rh, air pressure, '''
import numpy as np
import netCDF4 as nc
import matplotlib
import matplotlib.pyplot as plt
import datetime
import os
import sys
from scipy import stats
# from metpy import calc
path_nhchi = os.path.expanduser('~/Documents/projects/analytical-tools/py_nhchi/')
# adding to the system path
sys.path.insert(0, path_nhchi)
from convert_dir import vector_dir_to_from
from values_to_colors import truncate_colormap
from convert_time import datetime_to_yearday
from coare36vn_zrf_et import coare36vn_zrf_et

In [2]:
''' SD data info '''
year = '2023'
platfs_num = ['1031','1036','1040','1041','1042','1045','1057','1064','1065','1068','1069','1083']#['1036','1041','1069']
path_SD = os.path.expanduser('~/Documents/projects/sd-ni-wp/SD_windstress/compare_SD_model/data_timeseries_match_SD_GFS/')
filenames_SD_all = np.sort( os.listdir(path_SD) )
filenames_SD = np.array([])
for file in filenames_SD_all:
    if ('.txt' in file) & (year in file) & (file[-12:-8] in platfs_num):
        filenames_SD = np.append(filenames_SD, file)
print(len(filenames_SD), filenames_SD)

12 ['timeseries_wnd_match_2023-SD1031_GFS.txt'
 'timeseries_wnd_match_2023-SD1036_GFS.txt'
 'timeseries_wnd_match_2023-SD1040_GFS.txt'
 'timeseries_wnd_match_2023-SD1041_GFS.txt'
 'timeseries_wnd_match_2023-SD1042_GFS.txt'
 'timeseries_wnd_match_2023-SD1045_GFS.txt'
 'timeseries_wnd_match_2023-SD1057_GFS.txt'
 'timeseries_wnd_match_2023-SD1064_GFS.txt'
 'timeseries_wnd_match_2023-SD1065_GFS.txt'
 'timeseries_wnd_match_2023-SD1068_GFS.txt'
 'timeseries_wnd_match_2023-SD1069_GFS.txt'
 'timeseries_wnd_match_2023-SD1083_GFS.txt']


In [3]:
''' load all SD data in filenames_SD and plot stats '''
if 'data_all' in locals():
    del data_all
for i, platf_num in enumerate( platfs_num ):
    temp = np.genfromtxt( path_SD+'timeseries_wnd_match_'+year+'-SD'+platf_num+'_GFS.txt', dtype=float,\
                         delimiter=' ',skip_header=1)
    temp = np.column_stack( (i*np.ones( (temp.shape[0],) ), temp) )
    ''' append to data_all '''
    if 'data_all' not in locals():
        data_all = temp
    else:
        data_all = np.vstack( (data_all, temp) )
    print(platf_num, data_all.shape)

1031 (1968, 10)
1036 (4008, 10)
1040 (5094, 10)
1041 (6510, 10)
1042 (7034, 10)
1045 (9074, 10)
1057 (10532, 10)
1064 (11060, 10)
1065 (12068, 10)
1068 (13730, 10)
1069 (15938, 10)
1083 (17888, 10)


In [ ]:
''' plot 2D-histogram of uwnd & vwnd - individual SD '''
nrow = 3; ncol = 4
bins = np.arange(-1.25,1.25,0.05)
xlim = [-1.2,1.2]
x0 = np.arange(bins[0],bins[-1]+0.1,0.1)
histlim = [0,0.03]
vars_to_plot = ['uwnd','vwnd']
''' plot '''
for v, var_to_plot in enumerate( vars_to_plot ):
    plt.clf()
    plt.rcParams.update({'font.size': 13})
    fig, ax = plt.subplots(nrows=nrow,ncols=ncol)
    fig.set_size_inches(20, 14.5)
    for i, platf_num in enumerate(platfs_num):
        irows = np.where( data_all[:,0] == i )[0]
        print('Shape of',platf_num,var_to_plot,':',data_all[irows,:].shape)
        ''' plot '''
        plt.subplot(nrow,ncol,i+1)
        x = data_all[irows,4+v*2]
        y = data_all[irows,5+v*2]
        slope, intercept, r, p, std_err = stats.linregress(x,y)
        h, xedges,yedges = np.histogram2d(x, y, bins=bins,density=True)
        cs = plt.pcolormesh(xedges, yedges, h/h.sum(), cmap='turbo',vmin=histlim[0],vmax=histlim[1])
        plt.plot( x0, x0*slope+intercept, 'm--')
        plt.xlabel(var_to_plot+'$_{SD'+platf_num+'}$ (m s$^{-1}$)')
        plt.ylabel(var_to_plot+'$_{RTOFS}$ (m s$^{-1}$)')
        plt.title('y={:.2f}x+{:.2E}. r= {:.2f},\np= {:.2f}, std_err= {:.3f}, {:.0f} pairs'\
                  .format(slope,intercept,r,p,std_err,len(irows)), fontsize=12)
        # break
    ''' add colorbar '''
    axf = fig.add_axes([0.91,0.15,0.005,0.7])
    axf = plt.colorbar(cs,orientation='vertical',cax=axf,extend='max')
    # axf.set_label('',fontsize=14)
    ''' settings '''
    for i in range(nrow*ncol):
        plt.subplot(nrow,ncol,i+1)
        # plt.axis('equal')
        plt.plot(xlim,xlim,'--',color='lightgray')
        plt.grid()
        plt.xticks(np.arange(xlim[0],xlim[1],0.4))
        plt.yticks(np.arange(xlim[0],xlim[1],0.4))
        plt.xlim(xlim)
        plt.ylim(xlim)
    plt.suptitle('2D histogram of '+var_to_plot+' between \nSD$_{6m}$ ('+','.join(platfs_num)+') & RTOFS')
    plt.subplots_adjust(left=0.1,bottom=0.1,right=0.9,top=0.9,wspace=0.3,hspace=0.4)
    ''' save figure '''
    break
    plt.savefig('2Dhist_'+var_to_plot+'_SD(6m)('+year+':'+'-'.join(platfs_num)+')_GFS.png',dpi=400,bbox_inches='tight')